In [8]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import fix_yahoo_finance
import random
from scipy import stats
from functools import reduce
from operator import mul
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import math
from datetime import datetime
! pip install quandl
import quandl
from collections import defaultdict
import csv
from IPython.display import display

# to download csv preprocessed files later
from IPython.display import HTML
pointer = 0
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [9]:
####################################################
#      A. data preprocessed by investing.com       #
####################################################

##############################
#   1. Check data's format   #
##############################
# > 91 stocks were selected related to technology, along with 8 criteria: "Market Cap","P/E Ratio","Revenue","Average Vol. (3m)","EPS","Beta","Dividend","Yield"
# > However the parsing wasn't accurate and led to the following .csv data file which needs to be preprocessed

stocks = pd.read_csv("fundamental.csv", delimiter=',')
# to be able to see the whole names
pd.set_option('display.max_colwidth', -1)
# to check the 5 columns
stocks.head()

,"Name,""Market Cap"",""P/E Ratio"",""Revenue"",""Average Vol. (3m)"",""EPS"",""Beta"",""Dividend"",""Yield"""
0,"Apple,""912.44B"",""17.01"",""258.49B"",""29.29M"",""11.67"",""1.23"",""3.08"",""1.55%"""
1,"Microsoft,""1.03"",""30.14"",""122.21B"",""23.69M"",""4.48"",""1.23"",""1.84"",""1.36%"""
2,"Intel,""213.24B"",""10.86"",""70.84B"",""23.77M"",""4.36"",""0.83"",""1.26"",""2.66%"""
3,"Cisco,""240.92B"",""20.47"",""51.32B"",""20.06M"",""2.74"",""1.19"",""1.40"",""2.50%"""
4,"Broadcom,""110.29B"",""33.95"",""21.31B"",""2.79M"",""8.19"",""0.91"",""10.60"",""3.81%"""


In [0]:
########################################################################################
#   2. Preprocess csv files & Combine fundamental, technical and performance analysis  #
########################################################################################
# Step 1: For all 91 stocks extract the criteria results.
fundamental = open("fundamental.csv",mode= 'r')
fundamental = csv.reader(fundamental, delimiter=',')
fields_fundamenttal = next(fundamental)[0].split(',')

performance = open("performance.csv",mode= 'r')
performance = csv.reader(performance, delimiter=',')
fields_performance = next(performance)[0].split(',')

technical = open("technical.csv",mode= 'r')
technical = csv.reader(technical, delimiter=',')
technical_dic = {"Strong Sell": 0, "Sell": 1, "Neutral": 2, "Buy": 3, "Strong Buy": 4}
fields_technical = next(technical)[0].split(',')


# Step 2: Choose the criteria necessary for our analysis
# > choose performance criteria: YTD, 1 Year, 3 Year
# > choosse technical criteria: Weekly, Monthly
fields = fields_fundamenttal + fields_performance[-3:] + fields_technical[-2:]
fields = [fields[0]] + [i[1:-1] for i in fields[1:]]


# Step 3: parse simutanously all the stocks, keep the criteria results necessary,
# combine them, and create a new whole analysis stock csv with all the criteria
# combined

stocksCSV = open("stocks.csv", mode= 'w')
stocksCSV = csv.DictWriter(stocksCSV, fieldnames = fields)
stocksCSV.writeheader()

for f, p, t in zip(fundamental, performance, technical):
  ##################### FUNDAMENTAL #####################
  f = f[0].split(",")
  stock_name = f[0]
  f = [i[1:-1] for i in f[1:]]
  
  # edit 'Market Cap'
  if f[0][-1] == "B":
    f[0] = float(f[0][:-1]) * 10**9
  elif f[0][-1] == "M":
    f[0] = float(f[0][:-1]) * 10**6
  elif f[0][-1] == "K":
    f[0] = float(f[0][:-1]) * 10**3
  else :
    f[0] = float(f[0][:-1]) * 10**12
    
  # edit 'Revenue'
  if f[2][-1] == "B":
    f[2] = float(f[2][:-1]) * 10**9
  elif f[2][-1] == "M":
    f[2] = float(f[2][:-1]) * 10**6
  elif row[2][-1] == "K":
    f[2] = float(f[2][:-1]) * 10**3
  else :
    f[2] = float(f[2][:-1])
    
  # edit 'Average Vol.' 
  if f[3][-1] == "M":
    f[3] = float(f[3][:-1]) * 10**6
  elif f[3][-1] == "K":
    f[3] = float(f[3][:-1]) * 10**3
  else :
    f[3] = float(f[3][:-1])
    
  # edit 'Yield' : convert percentage to decia
  f[-1] = float(f[-1][:-1]) / 10
  
  # edit rest fields as floats
  f = list(map(float, f))
  
  
  ##################### PERFORMANCE #####################
  p = p[0].split(",")
  p = [i[1:-1] for i in p[4:]]
  p = list(map(float, p))
  
  ##################### TECHNICAL #####################
  t = t[0].split(",")
  t = [i[1:-1] for i in t[4:]]
  t = [technical_dic[i] for i in t]

  row_values = [stock_name] + f + p + t
  row_dic = {}
   
  for i in range(len(row_values)):
    row_dic[fields[i]] = row_values[i]
  
  stocksCSV.writerow(row_dic)
  
 

In [11]:
################################################
#   3. Read stocks & criteria as a dataframe   #
################################################

stocks_investingcom = pd.read_csv("stocks.csv")

# Step 2.1 : drop unecessary criteria
del stocks_investingcom['Dividend']
del stocks_investingcom['Yield']
fields.remove('Dividend')
fields.remove('Yield')

# Step 2.2 : Handle exceptions :drop stocks for which no info are provided by DM2 yahoo finance later
# EXCLUDE : Simulations Plus , Maxim
stocks_investingcom.index = range(91)
stocks_investingcom = stocks_investingcom.drop(stocks_investingcom.index[[34]])
stocks_investingcom = stocks_investingcom.drop(stocks_investingcom.index[[87]])
stocks_investingcom.index = range(89)

 

# to be able to see the whole names
pd.set_option('display.max_colwidth', -1)
display(stocks_investingcom.head())

# download stocks investing.com
stocks_investingcom.to_csv('a.csv')
create_download_link(filename = 'a.csv')



,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
0,Apple,9.124400e+11,17.01,2.584900e+11,29290000.0,11.67,1.23,25.83,6.42,108.71,4,4
1,Microsoft,1.000000e+12,30.14,1.222100e+11,23690000.0,4.48,1.23,32.35,31.96,168.48,4,4
2,Intel,2.132400e+11,10.86,7.084000e+10,23770000.0,4.36,0.83,0.53,-11.75,46.66,0,2
3,Cisco,2.409200e+11,20.47,5.132000e+10,20060000.0,2.74,1.19,29.59,28.37,94.97,4,4
4,Broadcom,1.102900e+11,33.95,2.131000e+10,2790000.0,8.19,0.91,7.86,4.99,74.77,2,4


In [12]:
########################
#   4. Normalization   #
########################
#############################################
#                        x_ij               #
#    r_ij = ---------------------------     #
#            sqrt(sum(x_ij) for all j)      #
#############################################
# In order to be able to compare different kinds of criteria the first step 
# is to make them dimensionless
stocks_investingcom_normalized = stocks_investingcom.copy()
for criterion in fields[1:]:
  crit_values = list(stocks_investingcom_normalized[criterion])
  rms = np.sqrt(sum([i**2 for i in crit_values]))
  stocks_investingcom_normalized[criterion] = stocks_investingcom_normalized[criterion] / rms
  
display(stocks_investingcom_normalized.head())

,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
0,Apple,0.646330,0.024870,0.848301,0.502343,0.377369,0.111103,0.093362,0.022484,0.123725,0.153393,0.139686
1,Microsoft,0.708354,0.044067,0.401063,0.406299,0.144868,0.111103,0.116928,0.111932,0.191751,0.153393,0.139686
2,Intel,0.151049,0.015878,0.232479,0.407671,0.140988,0.074972,0.001916,-0.041152,0.053105,0.000000,0.069843
3,Cisco,0.170657,0.029929,0.168420,0.344042,0.088602,0.107490,0.106952,0.099359,0.108087,0.153393,0.139686
4,Broadcom,0.078124,0.049638,0.069934,0.047850,0.264837,0.082198,0.028410,0.017476,0.085097,0.076696,0.139686


In [13]:
#################################
#   5. Weigthed Normalization   #
#################################
# Step 1: We define the weights for each criterion
# Market Cap : 2.5%
# P/E Ratio: 2.5%
# Revenue: 20%
# Average Vol: 2.5%
# EPS: 20%
# Beta: 10%
# YTD: 10%
# 1 Year: 10%
# 3 Year: 2.5%
# Weekly: 7.5%
# Monthly: 12.5%
weights = [ 0.025, 0.025, 0.1, 0.025, 0.1, 0.2, 0.1, 0.1, 0.025, 0.2, 0.1]
for i in range(1,len(fields)-1):
  stocks_investingcom_normalized[fields[i]] = stocks_investingcom_normalized[fields[i]] / weights[i]
  
display(stocks_investingcom_normalized.head())


,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
0,Apple,25.853210,0.248700,33.932024,5.023426,1.886843,1.111026,0.933620,0.899380,0.618626,1.533930,0.139686
1,Microsoft,28.334148,0.440671,16.042526,4.062990,0.724341,1.111026,1.169284,4.477286,0.958753,1.533930,0.139686
2,Intel,6.041974,0.158782,9.299178,4.076710,0.704939,0.749717,0.019157,-1.646061,0.265524,0.000000,0.069843
3,Cisco,6.826263,0.299288,6.736785,3.440421,0.443012,1.074895,1.069524,3.974362,0.540437,1.533930,0.139686
4,Broadcom,3.124973,0.496376,2.797367,0.478503,1.324186,0.821979,0.284098,0.699051,0.425487,0.766965,0.139686


In [14]:
########################
#   6. Final Ranking   #
########################
# Final ranking for the stocks of the portfolio
# Total sum per row: 
stocks_investingcom_normalized['Score'] = stocks_investingcom_normalized.sum(1)
stocks_investingcom_normalized = stocks_investingcom_normalized.sort_values('Score', ascending=False)
display(stocks_investingcom_normalized.head())


# download normalized stocks investing.com
stocks_investingcom_normalized.to_csv('b.csv')
create_download_link(filename = 'b.csv')

,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly,Score
0,Apple,25.853210,0.248700,33.932024,5.023426,1.886843,1.111026,0.933620,0.899380,0.618626,1.53393,0.139686,72.180471
1,Microsoft,28.334148,0.440671,16.042526,4.062990,0.724341,1.111026,1.169284,4.477286,0.958753,1.53393,0.139686,58.994641
3,Cisco,6.826263,0.299288,6.736785,3.440421,0.443012,1.074895,1.069524,3.974362,0.540437,1.53393,0.139686,26.078603
72,Universal Display,0.245940,1.538838,0.043999,0.125617,0.282946,1.363943,3.491226,15.290856,0.938210,1.53393,0.139686,24.995191
80,AudioCodes,0.013101,0.494621,0.023679,0.035071,0.075991,0.541964,2.140133,16.054348,1.608905,1.53393,0.139686,22.661429


In [15]:
#######################################
#      B. data preprocessed by me     #
#######################################
#####################################################
#   1. Asssign ticker symbols  to the company names #
#####################################################
# So far we have 2 corpora:
#      1) (Corpus s) Investing.com: Company names, Market Cap, P/E,...
#      2) (Corpus t) Yahoo Finance: Company names, ticker symbols resspectively
# For each company name in corpus s, we will find the most probable 
# company name matching in corpus t, and we will keep each ticker symbol
# eg. (s) 'Apple' ---- best matching ---> (t) 'Apple Inc.'' ---> Keep 'AAPL'

# Step 1 : Read corpus t as dataframe
tickers = pd.read_csv("companylist.csv")
tickers = tickers[['Symbol','Name']]
stocks_tickers = pd.DataFrame(columns=['Name', 'Symbol'])


# Step 2 : Define replacement dictionaries used in the tokenization process later
replacement_dic = {".": "", "&": " ", ",": " "}
replacement_bussiness_dic = {"corporation": "corp corpo","laboratories":"labs","inc":" "}

# Step 3: Define functions used in the tokenization process
def tokenize(s):
  # Step 3.1 : Python is capital sensitive, so lower the strings to find the ticker.
  s = s.lower()
  # Step 3.2 : replace common puncutation marks AND business abbreviations
  s = replace_all(s, replacement_dic)
  s = replace_all(s, replacement_bussiness_dic)
  # Step 3.3 : Tokenize company name by breaking down into words. 
  s = s.split(" ")
  s = [i for i in s if i != '']
  return s

def replace_all(text, dic):
    for i, j in dic.items():
      text = text.replace(i, j)
    return text

def any_2(list1, list2):
  # Step 3.4 : Check if at least 2 stock tokensis part of the ticker companies tokens. 
  c = 0
  for i in list1:
    if i in list2:
      c+=1
  length = len(list1)
  if c > 1 or (length == 1 and c == 1) : return True
  else: return False

  
# Ticker Assigment process:
# (s) ----------------------------...---------------->
#       |        |
#     Apple  Microsoft
# 1. Create word tokens for the company names of s,t
#    eg. s : 'Kulicke&Soffa'  -> ['kulicke', 'soffa']
#        t : 'Kulicke and Soffa Industries, Inc.' -> ['kulicke', 'and', 'soffa', 'industries']
# 2. Create matching list between the s and t word tokens iff only at least 2 s word tokens
#    exist in the t word tokens
# 3. For the non empty matching lists, find the respective ticker of the t company name
#    and append (company name, ticker) in the new dataframe
# 4. Manually add the tickers for the excpeptions of the previous process
exceptions = []
stock_ptr = 0
for s in stocks_investingcom_normalized['Name']:
  matching = [t for t in tickers['Name'] if any_2(tokenize(s),tokenize(t))]
  if matching and s != "Bel Fuse A" and s != "Bel Fuse B":
    symbol = tickers.loc[tickers['Name'] == matching[0], 'Symbol'].item()
    stocks_tickers.loc[stock_ptr] = [s, symbol]
    stock_ptr+=1
  else:
    exceptions.append(s)

exceptions
stocks_tickers.loc[stock_ptr] = [exceptions[0], 'SSNC']
stocks_tickers.loc[stock_ptr + 1] = [exceptions[1], 'CTSH']
stocks_tickers.loc[stock_ptr + 2] = [exceptions[2], 'BELFA']
stocks_tickers.loc[stock_ptr + 3] = [exceptions[3], 'BELFB']

stocks_tickers.index = range(89)
print(stocks_tickers)


# download stock tickers
stocks_tickers.to_csv('c.csv')
create_download_link(filename = 'c.csv')

                         Name Symbol
0   Apple                      AAPL 
1   Microsoft                  MSFT 
2   Cisco                      CSCO 
3   Universal Display          OLED 
4   AudioCodes                 AUDC 
5   Intel                      INTC 
6   Taitron                    TAIT 
7   Qualcomm                   QCOM 
8   Xilinx                     XLNX 
9   Bruker                     BRKR 
10  Sapiens                    SPNS 
11  Ubiquiti                   UBNT 
12  Cypress                    CY   
13  Intuit                     INTU 
14  CDW Corp                   CDW  
15  Equinix                    EQIX 
16  Broadcom                   AVGO 
17  Elbit Systems              ESLT 
18  Texas Instruments          TXN  
19  Garmin                     GRMN 
20  AstroNova                  ALOT 
21  Formula Systems ADR        FORTY
22  Teradyne                   TER  
23  Analog Devices             ADI  
24  Cerner                     CERN 
25  TESSCO                     TESS 
2

In [16]:
################################################
#   2. Read stocks & criteria as a dataframe   #
################################################
# Pull Adjusted closing prices with Pandas datareader
stocks_yahoofinance = pd.DataFrame()

for item in stocks_tickers['Symbol']:
  print(item)
  stocks_yahoofinance[item] = web.DataReader(item, data_source='yahoo', start='20-06-2016', end='20-06-2019')['Adj Close']

stocks_yahoofinance.head()

AAPL
MSFT
CSCO
OLED
AUDC
INTC
TAIT
QCOM
XLNX
BRKR
SPNS
UBNT
CY
INTU
CDW
EQIX
AVGO
ESLT
TXN
GRMN
ALOT
FORTY
TER
ADI
CERN
TESS
TTEC
SLP
AMAT
XPER
MANT
CSGS
AVT
KLAC
OTEX
CONE
MTSC
NVDA
NXPI
JKHY
CGNX
PRGS
JCOM
KE
AMSWA
STX
CCMP
TACT
CMTL
MCHP
MLAB
RFIL
MPWR
SABR
FELE
LOGI
BLKB
KLIC
POWI
DOX
CTXS
MGIC
GILT
NTAP
EGOV
NATI
IDCC
MNDO
SWKS
APWC
SIMO
LMNX
MKSI
HOLI
LFUS
ATVI
WDC
CDK
WSTG
TYPE
CPSI
AMOT
LOGM
EBIX
HIMX
SSNC
CTSH
BELFA
BELFB


,AAPL,MSFT,CSCO,OLED,AUDC,INTC,TAIT,QCOM,XLNX,BRKR,SPNS,UBNT,CY,INTU,CDW,EQIX,AVGO,ESLT,TXN,GRMN,ALOT,FORTY,TER,ADI,CERN,TESS,TTEC,SLP,AMAT,XPER,MANT,CSGS,AVT,KLAC,OTEX,CONE,MTSC,NVDA,NXPI,JKHY,...,MCHP,MLAB,RFIL,MPWR,SABR,FELE,LOGI,BLKB,KLIC,POWI,DOX,CTXS,MGIC,GILT,NTAP,EGOV,NATI,IDCC,MNDO,SWKS,APWC,SIMO,LMNX,MKSI,HOLI,LFUS,ATVI,WDC,CDK,WSTG,TYPE,CPSI,AMOT,LOGM,EBIX,HIMX,SSNC,CTSH,BELFA,BELFB
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-20,90.507019,47.147133,26.153543,69.143028,3.915042,29.689028,0.827579,48.072849,43.769341,23.988504,11.902412,39.695564,9.177250,104.200157,39.627918,351.233215,145.088516,87.840607,57.389999,37.275299,14.660143,30.346729,18.999050,53.227676,53.616055,10.634553,26.241299,6.880351,22.596193,28.421227,34.322407,38.577759,40.588165,67.999763,28.742767,47.027740,42.245632,46.984108,84.401352,80.731682,...,49.075775,121.214035,2.213694,66.441086,25.283728,33.259865,15.398465,65.642319,11.974093,51.760933,54.369980,67.102898,6.264921,4.416723,23.805387,19.382902,26.190271,55.395222,1.467397,63.015274,2.120881,42.053741,19.073784,40.243027,17.045143,114.527138,37.952457,43.261848,54.514122,16.014153,22.239342,36.251488,22.427084,59.721478,47.090622,8.953636,28.744833,59.739643,15.422829,17.929546
2016-06-21,91.277893,48.201756,26.126299,67.211601,3.895990,29.827463,0.827579,48.063927,43.938770,23.644127,12.076452,39.457691,9.042023,104.734024,39.926594,356.493591,144.053024,86.756165,57.417774,37.266533,14.232594,30.346729,19.213287,53.480743,54.673416,10.464671,26.174574,7.023493,22.787449,28.331089,34.454636,38.342178,40.815697,68.193649,29.200361,47.481934,41.620258,46.697613,84.915581,80.567612,...,49.113621,120.601700,2.177104,66.295349,24.707624,32.833710,15.445672,65.770065,12.042574,52.014229,54.054482,67.181717,6.367320,4.378729,23.814796,19.475645,26.115578,54.919899,1.467397,62.547085,2.092727,42.389725,19.200357,40.272137,17.240280,113.786118,37.923031,44.406773,54.582111,16.014153,22.388033,35.906933,22.011406,60.737118,46.972351,8.811666,28.568424,60.032009,15.403572,17.958401
2016-06-22,90.935295,48.013435,26.080891,66.853195,3.905516,29.799772,0.827579,48.206650,43.694046,23.870436,12.047445,39.418045,8.915815,103.326576,39.859169,356.625122,143.433594,87.070099,57.436287,37.082436,14.232594,30.714743,19.310671,53.283909,55.142239,10.558106,26.136446,6.928066,22.921324,28.502356,34.605751,38.445827,40.635571,68.599884,29.152693,47.664944,41.760262,46.658092,84.965019,80.471100,...,48.810856,119.959709,2.113071,66.237038,24.716915,33.182384,15.530643,65.583374,12.013225,51.682987,53.901516,67.079254,6.311466,4.378729,23.692520,19.484922,25.956852,53.180168,1.467397,62.441982,1.980114,42.380386,19.171150,40.291531,17.230522,113.864113,37.589512,43.934261,53.902149,16.014153,22.499556,35.385475,21.664999,60.967075,47.021629,8.830594,28.485121,60.148949,15.740524,18.112301
2016-06-23,91.458733,48.879723,26.534950,68.804527,3.962669,30.445789,0.811191,49.553661,44.654144,24.008188,12.182811,39.784767,9.456713,105.753212,40.206017,362.054535,146.651001,88.011848,58.583897,37.941555,14.394112,31.516813,19.748886,54.464874,56.748230,10.778951,26.422405,6.975779,23.447258,29.286573,35.011875,38.775642,41.043243,69.107697,29.395796,47.454483,42.441650,47.902843,86.389023,81.426559,...,49.993519,121.826355,1.975859,67.908272,24.921339,34.257473,15.794994,67.204536,12.189315,51.741447,54.044922,68.056564,6.423173,4.369231,24.473179,19.753872,26.395689,54.254421,1.481506,64.219200,2.167803,42.977692,19.472979,41.552769,18.040337,116.330940,38.178074,46.169594,54.426697,15.730489,22.936354,35.972118,21.932226,62.988766,47.908646,8.849524,28.950645,61.104008,16.077478,18.602863
2016-06-24,88.889130,46.921150,25.200026,65.100975,3.886464,29.116842,0.819385,46.493908,42.649223,22.532272,11.602676,38.248489,9.105129,102.239456,38.240505,353.149536,137.498016,85.015327,56.029537,35.741161,14.365611,31.035572,18.502405,51.587444,54.992615,10.889374,25.230

In [17]:
###################################################
#   3. Calculate daily & annualized/ytd returns   #
###################################################
from functools import reduce
from operator import mul

def daily(data):
  daily_returns = data.pct_change()
  daily_returns = daily_returns + 1
  daily_returns = daily_returns.fillna(0)
  return daily_returns


def annualized(data, tickers, daily_returns):
  d = {'Annualized Returns': ['1 Year', '2 Year', '3 Year', 'YTD']}
  annualized_returns = pd.DataFrame(d).set_index('Annualized Returns')
  length = data.shape[0]
  
  print()

  for item in tickers['Symbol']:
    ar = []
    # Year 1,2,3
    for i in range(251,length,251):
      ar.append((reduce(mul,daily_returns[item][(length-i):],1)**(251/i) - 1)*100)
    # YTD
    
    ytd_length = data[data.index >= '01-01-2019'].shape[0]
    ar.append( ( reduce(mul, daily_returns[item][(length - ytd_length):], 1) - 1)*100)
    annualized_returns[item] = pd.Series(ar, index = annualized_returns.index)
  
  return annualized_returns


#######################DAILY RETURNS#######################
# Simple daily returns by using 'pct_change()' funciton (Percentage change between the current and a prior element.)
daily_returns = daily(stocks_yahoofinance)


#######################ANNUALIZED & YTD RETURNS#######################
# Annualized Return's Difference From Average Return
# Calculations of simple averages only work when numbers are independent of each other. The annualized return is used because the amount of investment lost or 
# gained in a given year is interdependent with the amount from the other years under consideration because of compounding. For example, if a mutual fund manager loses half of 
# her client's money, she has to make a 100% return to break even. Using the more accurate annualized return also gives a clearer picture when comparing various mutual funds or 
# the return of stocks that have traded over different time periods. 
# However, when we want to know the average of annual returns that are compounded, the simple average is not accurate
# Note: Annualise daily returns ~> 250 trading days in a year

annualized_returns = annualized(stocks_yahoofinance, stocks_tickers, daily_returns)

display(annualized_returns.head())


# download stock from yahoo finance
stocks_yahoofinance.to_csv('d.csv')
create_download_link(filename = 'd.csv')

  

,AAPL,MSFT,CSCO,OLED,AUDC,INTC,TAIT,QCOM,XLNX,BRKR,SPNS,UBNT,CY,INTU,CDW,EQIX,AVGO,ESLT,TXN,GRMN,ALOT,FORTY,TER,ADI,CERN,TESS,TTEC,SLP,AMAT,XPER,MANT,CSGS,AVT,KLAC,OTEX,CONE,MTSC,NVDA,NXPI,JKHY,...,MCHP,MLAB,RFIL,MPWR,SABR,FELE,LOGI,BLKB,KLIC,POWI,DOX,CTXS,MGIC,GILT,NTAP,EGOV,NATI,IDCC,MNDO,SWKS,APWC,SIMO,LMNX,MKSI,HOLI,LFUS,ATVI,WDC,CDK,WSTG,TYPE,CPSI,AMOT,LOGM,EBIX,HIMX,SSNC,CTSH,BELFA,BELFB
Annualized Returns,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1 Year,8.577046,36.619081,35.050945,111.850844,125.713584,-9.515439,100.756742,28.845209,69.827261,61.278094,61.315859,58.656429,31.420083,25.910058,29.775316,26.525416,9.733407,34.259974,1.492692,36.988390,40.324888,32.228869,18.920385,13.788578,17.800925,25.292761,24.037340,28.368191,-9.221590,13.700024,20.597693,15.953944,5.434070,6.661962,17.540493,9.054182,4.680319,-41.033900,-12.228474,6.864911,...,-13.942794,9.368726,2.087025,-6.824284,-8.940709,2.380162,-9.513577,-25.516060,-3.102424,-5.964656,-8.572195,-7.593797,4.941403,3.458110,-18.030365,-0.820664,-5.681401,-21.987878,10.119682,-22.696587,-11.130761,-18.833955,-30.624856,-26.212716,-16.114192,-22.756586,-40.087368,-48.583682,-24.215445,-13.875680,-16.827837,-19.281590,-26.705475,-34.662435,-40.152424,-56.065517,8.188270,-17.636982,-29.228781,-22.380219
2 Year,18.733969,42.099427,38.497430,25.778624,61.669828,19.814989,47.083510,18.207489,34.166345,31.263182,21.385485,66.064812,33.608840,39.146974,32.762841,10.227479,9.476023,14.750963,22.091591,32.273455,41.168892,12.686367,20.225752,21.115761,3.497471,22.399777,8.120352,55.556931,1.043818,-16.233798,31.759313,12.170681,9.177737,10.671194,15.615187,5.469159,4.049154,-1.376552,-4.718056,16.250315,...,4.678138,26.412268,118.975366,16.652364,3.660482,11.367949,5.252514,-2.514953,5.636393,9.734286,-0.289862,11.558450,6.349192,41.417188,29.870620,-7.538996,3.394344,-8.953088,4.392311,-12.945487,-12.328685,-5.843119,1.486973,3.012902,6.667413,6.497530,-12.673374,-31.917050,-10.299281,-18.961844,-1.012528,-7.514411,14.002401,-17.765366,-6.717962,-33.297496,24.237226,-1.645898,-17.043819,-15.692833
3 Year,29.929505,41.818563,30.083349,40.619199,59.205938,16.558963,57.445808,14.697894,37.971514,26.778148,11.366041,50.887023,35.517235,37.166676,39.458785,12.544081,24.440682,22.372827,25.196454,30.143393,22.197584,17.242780,33.746227,28.310638,9.172311,21.527454,20.147038,60.061326,23.532582,-11.764373,23.192800,8.251648,2.515561,18.375068,12.555602,8.433477,10.136445,48.921897,4.876152,19.875007,...,20.582779,23.426636,53.012620,25.305480,-2.371221,12.240639,36.616110,7.647564,23.064133,12.826372,4.984436,13.999668,10.906662,24.794034,38.298077,-6.201523,15.822270,6.847591,13.929814,6.694133,3.572428,0.258580,3.344927,23.643435,3.130657,16.452920,6.705819,-3.370417,-2.780344,-10.632263,-7.564705,-8.869899,17.449041,5.979188,0.090920,-27.465160,27.211185,2.180493,-2.923429,-1.994546
YTD,27.479941,35.910181,34.366063,98.907035,60.766607,1.811100,90.680438,30.483966,35.627288,63.711698,50.861286,36.729617,76.609592,36.021901,34.175385,45.772452,10.721518,40.836914,20.975974,30.837133,39.372281,38.888891,47.866843,32.544434,37.164853,61.851019,59.994754,43.600526,33.332760,8.363096,24.875809,55.108744,22.446418,28.932597,28.586478,15.945257,40.566435,15.681567,34.466905,10.217149,...,20.001686,8.382556,4.579837,12.435370,7.643844,10.085730,26.598458,30.467811,11.063502,22.319420,7.035003,-2.317207,12.844031,-2.189168,6.315836,30.092051,-10.095290,-1.336945,6.738814,14.301267,13.402061,25.659151,-7.955790,18.468841,7.938317,5.371064,-1.144808,8.416606,4.028845,17.574055,15.168424,7.405964,-21.326538,-10.332798,11.088063,-1.749276,30.434097,1.703043,5.421110,-6.831252


In [18]:
############################
#   5. Calculate metrics   #
############################
def apr(data, r_f):
  apr = (np.log(data) - np.log(data.shift(1)))*252*100
  apr -= r_f
  apr = apr.to_frame()
  apr.fillna(0,inplace=True)
  return apr

# We will work based on CAMP model:
########################################
#     R_s = R_f + beta * (R_m - R_f)   #
########################################
# R_s : Expected return of the securiy
# R_f : Risk-free rat
# R_m : Expected return of the market chosen

# Step 1 : Risk free rate based on [ Tbills ]
# This asset exists only in theory but often yields on low-risk instruments like 3-month U.S. 
# Treasury Bills can be viewed as  being virtually risk-free and thus their yields can be used 
# to approximate the risk-free rate. I get the data for these instruments below.
# KEY NOTE: we won't get the most recent Treasury Bill rate, but instead we will use whole historic 
# data, so that our calculations are more precise
tbill = pd.DataFrame()
tbill = web.DataReader('^IRX', data_source='yahoo', start='20-06-2016',end = '20-06-2019')['Adj Close']

# Step 2 : Market retun based on [S&P500]
market = pd.DataFrame({'^GSPC' : web.DataReader('^GSPC', data_source='yahoo', start='20-06-2016',end = '20-06-2019')['Adj Close']})
market_ticker = pd.DataFrame(columns = ['Symbol'])
market_ticker.loc[0] = ['^GSPC']


# Step 3 : 1) Calculate [ annual percentage rate ] of the market, so the amount of interest an investment earns over the course of a year
#          2) Daily returns of the market
#          3) Annualized returns of the markett
market_apr = apr(market['^GSPC'], tbill)['^GSPC']
market_daily_returns = daily(market)
market_annualized_returns = annualized(market, market_ticker, market_daily_returns)
# std, mean 
market_std = market_apr.std()
market_mean = market_apr.mean()



# Step 4 : Define stocks_yahoofinance dataframe & calculate some metrics
stocks_yahoofinance_edited = pd.DataFrame(columns = ['alpha', 'beta', 'r-squared', 'share_ratio', 'treynor_ratio', 'f_test'])
stock_ptr = 0


for item in stocks_tickers['Symbol']:
  # Step 4.1 : Calculate [ annual percentage rate ] of each stock
  stock_apr = apr(stocks_yahoofinance[item], tbill)
  
  # Step 4.2 : How much each stock is correlated with Market Benchmark (our approximation of the market).
  smcorr = stock_apr.corrwith(market_apr).item()
  
  
  # Step 4.3 : Calculate alpha and beta
  # std, mean
  stock_std = stock_apr.std().item()
  stock_mean = stock_apr.mean().item()
  
  # beta, alpha
  ########################################
  #                σ_Y        _       _  #
  #    b = r_XY * ----- , a = Y - b * X  #
  #                σ_X                   #
  ########################################
  
  beta = smcorr * stock_std / market_std
  alpha = stock_mean - beta * market_mean
  
  # Step 4.4 :  Calculate Annualised sharpe ratio
  sharpe_ratio = (stock_mean / stock_std) * math.sqrt(252)
  
  # Step 4.5: R^2
  
  ##############################################################################
	#                         _         _                                         #
	#              1     (yi- y)   (xi- x)                                       #
	#   R^2 =   (----- Σ(------- x -------))**2 or R^2 = (Σ(smcorr))**2          #
	#            (n-1)    σ(yi)     σ(xi)                                        #
  ##############################################################################
  r_squared = smcorr**2 * 100
  
  # Step 4.6 : Treynor Measure
  #################################
  #                  R_s - R_f    #
  #     treynor = -------------   #
  #                   beta        #
  #################################
  # > The Treynor ratio, also known as the reward-to-volatility ratio, is a performance 
  #   metric for determining how much excess return was generated for each unit of 
  #   risk taken on by a portfolio.
  # > R_s: We will use last 3 year annualized return, since we have an 
  #   long term horizon concept and we seek for low volatility
  # > R_f : 5%
  treynor_ratio = ( annualized_returns[item][2]/100 - 0.05) / beta
  
  
  # Step 4.7 : F-test
  #############################
  #                R^2        #
  #               -----       #
  #                k-1        #
  #          F = -------      #
  #               1-R^2       #
  #               -----       #
  #                n-k        #
  #############################
  # > k = 2 parameters of the CAMP model (alpa, beta)
  # > n = the length of the samples
  
  f_test = r_squared/100 / ( (1 - r_squared/100) / (stocks_yahoofinance.shape[0] - 2) )
 
  stocks_yahoofinance_edited.loc[stock_ptr] = [alpha, beta, r_squared , sharpe_ratio, treynor_ratio, f_test]
  stock_ptr+=1
  

# Step 5.1 : Droping stocks based on their bad sharpe ratio performance. The higher the better
stocks_yahoofinance_edited.index = stocks_tickers['Symbol']
stocks_yahoofinance_edited = stocks_yahoofinance_edited.sort_values(['share_ratio'], ascending=[False])
stocks_yahoofinance_edited = stocks_yahoofinance_edited[stocks_yahoofinance_edited['share_ratio'] >=0]


# Step 5.2 : Droping stocks based on their bad treynor_ratio performance. The higher the better
stocks_yahoofinance_edited = stocks_yahoofinance_edited.sort_values(['treynor_ratio'], ascending=[False])
stocks_yahoofinance_edited = stocks_yahoofinance_edited[stocks_yahoofinance_edited['treynor_ratio'] >=0]


# Step 5.3 : Droping stocks based on their alpha performance, since Alpha is one of the five major risk management 
# indicators for mutual funds, stocks and bonds, and in a sense tells investors whether an asset has performed 
# better or worse than its beta predicts.
stocks_yahoofinance_edited = stocks_yahoofinance_edited.sort_values(['alpha'], ascending=[False])
stocks_yahoofinance_edited = stocks_yahoofinance_edited[stocks_yahoofinance_edited['alpha'] > 0 ]

# Step 5.4 : Droping stocks based on their f_test performance. Based on that metric we can
# conclude upon the importance of the CAMP model and how much useful is it. The higher the better
# since we can guarantee about the accuracy of our results
stocks_yahoofinance_edited = stocks_yahoofinance_edited.sort_values(['f_test'], ascending=[False])
stocks_yahoofinance_edited = stocks_yahoofinance_edited[stocks_yahoofinance_edited['f_test'] >= 100]

print(stocks_yahoofinance_edited.shape)
stocks_yahoofinance_edited




(39, 6)


,alpha,beta,r-squared,share_ratio,treynor_ratio,f_test
Symbol,,,,,,
MSFT,19.814749,1.373946,64.113859,1.563936,0.267977,1347.089665
CSCO,11.454632,1.259557,57.955342,1.166749,0.199144,1039.331263
MPWR,3.799481,1.613388,47.494265,0.689570,0.125856,682.033608
TXN,7.024910,1.333638,47.415005,0.847315,0.151439,679.869105
INTU,16.673324,1.223769,46.375700,1.287254,0.262849,652.078958
AAPL,11.614751,1.274979,44.646467,1.024879,0.195529,608.153341
INTC,0.243013,1.330235,40.711661,0.529017,0.086894,517.750922
AMAT,2.311300,1.664746,39.746756,0.582717,0.111324,497.384900
ADI,10.066974,1.257403,39.541214,0.909721,0.185387,493.130561


In [19]:
#################################
#   6. Merge metrics & Returns  #
#################################
annualized_returns = annualized_returns.transpose()
annualized_returns.index.names = ['Symbol']
annualized_returns
stocks_yahoofinance_edited = pd.merge(stocks_yahoofinance_edited, annualized_returns, how='inner', on = 'Symbol')



# download stock from yahoo finance & metrics calculated
stocks_yahoofinance_edited.to_csv('e.csv')
create_download_link(filename = 'e.csv')

print(stocks_yahoofinance_edited.shape)
stocks_yahoofinance_edited


(39, 10)


,alpha,beta,r-squared,share_ratio,treynor_ratio,f_test,1 Year,2 Year,3 Year,YTD
Symbol,,,,,,,,,,
MSFT,19.814749,1.373946,64.113859,1.563936,0.267977,1347.089665,36.619081,42.099427,41.818563,35.910181
CSCO,11.454632,1.259557,57.955342,1.166749,0.199144,1039.331263,35.050945,38.497430,30.083349,34.366063
MPWR,3.799481,1.613388,47.494265,0.689570,0.125856,682.033608,-6.824284,16.652364,25.305480,12.435370
TXN,7.024910,1.333638,47.415005,0.847315,0.151439,679.869105,1.492692,22.091591,25.196454,20.975974
INTU,16.673324,1.223769,46.375700,1.287254,0.262849,652.078958,25.910058,39.146974,37.166676,36.021901
AAPL,11.614751,1.274979,44.646467,1.024879,0.195529,608.153341,8.577046,18.733969,29.929505,27.479941
INTC,0.243013,1.330235,40.711661,0.529017,0.086894,517.750922,-9.515439,19.814989,16.558963,1.811100
AMAT,2.311300,1.664746,39.746756,0.582717,0.111324,497.384900,-9.221590,1.043818,23.532582,33.332760
ADI,10.066974,1.257403,39.541214,0.909721,0.185387,493.130561,13.788578,21.115761,28.310638,32.544434


In [20]:
##########################################
#   7. LSTM & Stock movement prediciton  #
##########################################
# Step 1 : import necessary libraries
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)


def LSTM_model(batch_size, look_back):
  model = Sequential()
  model.add(  LSTM(4, batch_input_shape = (batch_size, look_back, 1), stateful = True, return_sequences = True)  ) 
  model.add(  LSTM(4, batch_input_shape = (batch_size, look_back, 1), stateful = True)  ) 
  model.add( Dense(1) )
  model.compile(Adam(lr=0.01), loss='mean_squared_error')
  return model

weekly = [0] * stocks_yahoofinance_edited.shape[0]
monthly = [0] * stocks_yahoofinance_edited.shape[0]
ptr = 0

# Step 2 : 
for item in stocks_yahoofinance_edited.index:
  # Step 2.0 :  fix random seed for reproducibility
  numpy.random.seed(7)
  
  # Step 2.1 : load data (adjusted closed prices) & normalize
  data = stocks_yahoofinance[item].values
  data = data.astype('float32')
  data = data.reshape(-1,1)
  scaler = MinMaxScaler(feature_range = (0,1))
  data = scaler.fit_transform(data)
  
  # Step 2.2 : split into train & test sets
  train_size = int(len(data) * 0.8)
  # we will analyse weekly & and montthly predictions to extract resulst
  # for our technical analysis (22 & 5 trading days respectively)
  test_size_monthly = 27
  test_size_weekly = 9
  train_data, test_data = data[0:train_size, :], data[train_size : len(data), :]
  
  # Step 2.3 : 
  # reshape into X=t and Y=t+1
  look_back = 4
  trainX, trainY = create_dataset(train_data, look_back)
  testX, testY = create_dataset(test_data, look_back)
  
  # reshape input to be [samples, time steps, features]
  trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
  testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

  # Step 2.4 : Create and fit the LSTM network
  batch_size = 1
  model = LSTM_model(batch_size, look_back)
  
  # Step 2.5 : Train model
  for i in range(10):
    model.fit(trainX, trainY, epochs = 1, batch_size = batch_size,verbose=0 ,shuffle = True)
    model.reset_states()
    
  # Step 2.6 : Make predictions
  trainPredict = model.predict(trainX, batch_size=batch_size)
  model.reset_states()
  testPredict = model.predict(testX, batch_size=batch_size)
  
  # Step 2.7 : invert predictions
  trainPredict = scaler.inverse_transform(trainPredict)
  trainY = scaler.inverse_transform([trainY])
  
  testPredict = scaler.inverse_transform(testPredict)
  testY = scaler.inverse_transform([testY])
  
  # Step 2.8 : calculate root mean squared error
  trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
  print('Train Score: %.2f RMSE' % (trainScore))
  
  testMonthlyScore = math.sqrt(mean_squared_error(testY[0][-test_size_monthly : ], testPredict[-test_size_monthly : ,0]))
  print('Monthly Test Score: %.2f RMSE' % (testMonthlyScore))
  testWeeklyScore = math.sqrt(mean_squared_error(testY[0][-test_size_weekly : ], testPredict[-test_size_weekly:,0]))
  print('Weekly Test Score: %.2f RMSE' % (testMonthlyScore))
  
  MonthlyME = np.mean( testPredict[-test_size_monthly:,0] - testY[0][-test_size_monthly:])
  WeeklyME = np.mean( testPredict[-test_size_weekly:,0] - testY[0][-test_size_weekly:])
  print('Weekly Mean Error: %.2f ' % (MonthlyME))
  print('Monly Mean Error: %.2f ' % (WeeklyME))
  
  weekly[ptr] = WeeklyME
  monthly[ptr] = MonthlyME
  
  # Step 2.9 : Shift predictions for plotting
  # shift train predictions for plotting
  trainPredictPlot = numpy.empty_like(data)
  trainPredictPlot[:, :] = numpy.nan
  trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
  
  # shift test predictions for plotting
  testPredictPlot = numpy.empty_like(data)
  testPredictPlot[:, :] = numpy.nan
  testPredictPlot[len(trainPredict)+(look_back*2)+1:len(data)-1, :] = testPredict
  
  # Step 2.10 : ploting baseline & monthly, weekly predictions
  plt.plot(scaler.inverse_transform(data))
  plt.plot(trainPredictPlot)
  plt.plot(testPredictPlot)
  plt.title("Timeseries & predictions of stock : " + item)
  plt.show()
  
  ptr+=1
  
  

  



Using TensorFlow backend.
W0627 23:50:21.317102 139944323762048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0627 23:50:21.353706 139944323762048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0627 23:50:21.359812 139944323762048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0627 23:50:21.779178 139944323762048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0627 23:50:21.954833 13994432

KeyboardInterrupt: ignored

In [21]:
#################################################################
#   8. Convertion of technical predictions into recomendations  #
#################################################################
# In this section, we use the arrays weekly, monthly that occured as the ME
# (mean error) of our predictions for last's week, month data, and after
# we normalize them between (-1,1) we classify the result as 'Strong Buy', 'Buy'
# ..., and more specifically as 0 (Strong Sell) to 4 (Strong Buy)

weekly = [-2.53,-1.69, 1.76, -0.39, -7.10, 2.4, 0.49,0.25, 
         -1.67, 1.43, 0.42, -0.06,-0.88, -0.55,
         1.87, 0.53,1.03,-1.29,-1.68,-0.89,-10.87,-3.67,
         -9.04,-1.34,-1.06,0.04,0.27,-1.27,-1.43,-0.14,
         -1.19,-1.89,-1.23,0.24,0.61,-7.55,-0.4,-15.0,
         0.03]
monthly = [-4.69,-2.22, 1.26, -0.67, -9.94, 1.51, 0.36, 0.25,
          -3.29, 1.1, 0.03, -0.39,-1.12, -0.97,
          1.44, 0.28,0.82,-1.86,-3.33,-1.07,-12.37,-5.35,
          -9.99, -2.19,-1.4,0.01,0.12,-1.53,-2.19,-0.23,
          -1.37,-4.18,-1.18,0.07,-0.76,-9.75,-1.27,-20.86,
          -0.03]


print(np.mean(weekly))
print(np.mean(monthly))

scaler = MinMaxScaler(feature_range = (-1,1))
weekly = np.array(weekly).reshape(-1,1)
monthly = np.array(monthly).reshape(-1,1)

weekly = scaler.fit_transform(weekly).reshape(1,-1)[0].tolist()
monthly = scaler.fit_transform(monthly).reshape(1,-1)[0].tolist()



for i in range(len(weekly)):
  # strong sell
  if weekly[i] <= -0.6:
    weekly[i] = 0
  # sell
  elif weekly[i] > -0.6 and weekly[i] <= -0.2:
    weekly[i] = 1
  # neutral
  elif weekly[i] > -0.2 and weekly[i] <= 0.2:
    weekly[i] = 2
  # buy
  elif weekly[i] > 0.2 and weekly[i] <= 0.6:
    weekly[i] = 3
  # strong buy
  else:
    weekly[i] = 4


for i in range(len(monthly)):
  # strong sell
  if monthly[i] <= -0.6:
    monthly[i] = 0
  # sell
  elif monthly[i] > -0.6 and monthly[i] <= -0.2:
    monthly[i] = 1
  # neutral
  elif monthly[i] > -0.2 and monthly[i] <= 0.2:
    monthly[i] = 2
  # buy
  elif monthly[i] > 0.2 and monthly[i] <= 0.6:
    monthly[i] = 3
  # strong buy
  else:
    monthly[i] = 4
    
stocks_yahoofinance_edited['Weekly'] = pd.Series(weekly, index=stocks_yahoofinance_edited.index)
stocks_yahoofinance_edited['Monthly'] = pd.Series(monthly, index=stocks_yahoofinance_edited.index)
stocks_yahoofinance_edited



# download stock from yahoo finance & metrics calculated by using LSTM
stocks_yahoofinance_edited.to_csv('f.csv')
create_download_link(filename = 'f.csv')


-1.6266666666666665
-2.4858974358974364


In [22]:
###########################################################################
#      C. DM1 (investing.com) & DM2 (preprocess yahoofinance) consensus   #
###########################################################################
# In this sectiton we calculate score for each DM for each stock, the total
# score of the 2 DM's for each score (along with average, standard deviation)
# and we exclude more stocks if they are not under a specific STD threshold s.
# HERE we prefer s = 0.18, so every stock with s > 0.18 is excluded
##################################################################################
#   1. DM2 results combined (our own Beta, YTD, weekly, monthly recommendations) #
##################################################################################
# Step 1 : changstocks_investingcome index of DM1
# 1.1
tickers = [stocks_tickers.loc[stocks_tickers['Name'] == i, 'Symbol'].item() for i in stocks_investingcom['Name']]
stocks_investingcom.index = tickers
stocks_investingcom.index.names = ['Symbol']
# 1.2
tickers_exclude = list(set(list(stocks_investingcom.index)) -  set(list(stocks_yahoofinance_edited.index)) )
stocks_investingcom = stocks_investingcom.drop(tickers_exclude)
stocks_investingcom = stocks_investingcom.reindex(stocks_yahoofinance_edited.index)

# Step 2 : define the new DM2 Dataframe
stocks_yahoofinance_edited2 = pd.DataFrame(columns = stocks_investingcom.columns, index = stocks_yahoofinance_edited.index )
for item in stocks_yahoofinance_edited.index:
  
  # Step 2.1 : select the respective rows froms invesing_com, and preprocessed yahoo finance
  # to update the values of beta, YTD, weekly, monthly based on ou finding from
  # the previous analysis
  
  stock_yahoofinance_row = stocks_yahoofinance_edited.loc[item].to_frame().transpose()
  stock_investitngcom_row = stocks_investingcom.loc[item].to_frame().transpose()
  
  # Step 2.3 : Update the values and append the new row in the new DM2 datatfame
  # change beta
  stock_investitngcom_row['Beta'] = stock_yahoofinance_row['beta']
  # change YTD
  stock_investitngcom_row['YTD'] = stock_yahoofinance_row['YTD']
  # change weekly
  stock_investitngcom_row['Weekly'] = stock_yahoofinance_row['Weekly']
  # change monthly
  stock_investitngcom_row['Monthly'] = stock_yahoofinance_row['Monthly']
  
 
  stock_investitngcom_row = stock_investitngcom_row.convert_objects(convert_numeric = True)
  stocks_yahoofinance_edited2.loc[item] = stock_investitngcom_row.squeeze()
  stocks_yahoofinance_edited2 = stocks_yahoofinance_edited2.convert_objects(convert_numeric = True)
  
display(stocks_yahoofinance_edited2)


# download combined results fromn DM1, DM2
stocks_yahoofinance_edited2.to_csv('g.csv')
create_download_link(filename = 'g.csv')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
Symbol,,,,,,,,,,,,
MSFT,Microsoft,1.000000e+12,30.14,1.222100e+11,23690000.0,4.48,1.373946,35.910181,31.96,168.48,3.0,3.0
CSCO,Cisco,2.409200e+11,20.47,5.132000e+10,20060000.0,2.74,1.259557,34.366063,28.37,94.97,3.0,4.0
MPWR,Monolithic,5.570000e+09,53.67,5.945900e+08,295460.0,2.43,1.613388,12.435370,-9.41,87.19,4.0,4.0
TXN,Texas Instruments,1.046900e+11,20.16,1.559000e+10,4850000.0,5.51,1.333638,20.975974,-2.90,78.89,4.0,4.0
INTU,Intuit,6.746000e+10,41.34,6.780000e+09,1470000.0,6.25,1.223769,36.021901,21.67,142.00,2.0,2.0
AAPL,Apple,9.124400e+11,17.01,2.584900e+11,29290000.0,11.67,1.274979,27.479941,6.42,108.71,4.0,4.0
INTC,Intel,2.132400e+11,10.86,7.084000e+10,23770000.0,4.36,1.330235,1.811100,-11.75,46.66,4.0,4.0
AMAT,Applied Materials,3.971000e+10,11.86,1.577000e+10,9230000.0,3.57,1.664746,33.332760,-12.91,79.05,4.0,4.0
ADI,Analog Devices,4.070000e+10,26.19,6.240000e+09,2780000.0,4.20,1.257403,32.544434,8.73,93.38,3.0,3.0


In [23]:
########################
#   2. Normalization  #
########################

def normalization(dm, decision, weights):
  dm_normalized = dm.copy()
  fields = list(dm_normalized.columns.values)
  
  # Step 1: In order to be able to compare different kinds of criteria the first step 
  # is to make them dimensionless
  for criterion in fields[1:]:
    crit_values = list(dm_normalized[criterion])
    rms = np.sqrt(sum([i**2 for i in crit_values]))
    dm_normalized[criterion] = dm_normalized[criterion] / rms
    
  for i in range(1,len(fields)-1):
    dm_normalized[fields[i]] = dm_normalized[fields[i]] / weights[i]
 
  # Step 3: Final ranking for the stocks of the portfolio
  # Total sum per row: 
  dm_normalized['DM' + str(decision)] = dm_normalized.sum(1)
  return dm_normalized

# Market Cap : 1 %
# P/E Ratio: 2.5%
# Revenue: 2.5%
# Average Vol: 1.5%
# EPS: 10%
# Beta: 25%
# YTD: 15%
# 1 Year: 6.5%
# 3 Year: 1%
# Weekly: 25%
# Monthly: 10%
weights = [ 0.01, 0.025, 0.025, 0.015, 0.1, 0.25, 0.15, 0.065, 0.01, 0.25, 0.1]
 
stocks_investingcom_normalized = normalization(stocks_investingcom,1, weights)
stocks_yahoofinance_normalized = normalization(stocks_yahoofinance_edited2,2,weights)

display(stocks_investingcom_normalized.head())
display(stocks_yahoofinance_normalized.head())

,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly,DM1
Symbol,,,,,,,,,,,,,
MSFT,Microsoft,28.367598,4.451415,26.918748,4.165252,0.700526,1.025163,2.406621,17.322940,0.991374,2.119996,0.185695,88.655328
CSCO,Cisco,6.834322,3.023240,11.304068,3.527014,0.428446,0.991825,2.201296,15.377091,0.558825,2.119996,0.185695,46.551817
MPWR,Monolithic,0.158008,7.926591,0.130968,0.051949,0.379973,1.375219,0.752116,-5.100403,0.513046,0.529999,0.092848,6.810312
TXN,Texas Instruments,2.969804,2.977456,3.433952,0.852743,0.861584,1.016829,1.293698,-1.571856,0.464207,2.119996,0.185695,14.604107
INTU,Intuit,1.913678,6.105557,1.493406,0.258460,0.977296,0.900143,2.378352,11.745561,0.835560,2.119996,0.185695,28.913704


,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly,DM2
Symbol,,,,,,,,,,,,,
MSFT,Microsoft,28.367598,4.451415,26.918748,4.165252,0.700526,1.123230,2.457891,17.322940,0.991374,1.383797,0.131306,88.014077
CSCO,Cisco,6.834322,3.023240,11.304068,3.527014,0.428446,1.029714,2.352203,15.377091,0.558825,1.383797,0.175075,45.993794
MPWR,Monolithic,0.158008,7.926591,0.130968,0.051949,0.379973,1.318979,0.851145,-5.100403,0.513046,1.845062,0.175075,8.250392
TXN,Texas Instruments,2.969804,2.977456,3.433952,0.852743,0.861584,1.090277,1.435711,-1.571856,0.464207,1.845062,0.175075,14.534014
INTU,Intuit,1.913678,6.105557,1.493406,0.258460,0.977296,1.000457,2.465537,11.745561,0.835560,0.922531,0.087538,27.805580


In [24]:
#####################################
#   3. Combine scores of DM1, DM2   #
#####################################
# Step 1 : Combine DM1, DM2 scores
final = pd.merge(stocks_investingcom_normalized,stocks_yahoofinance_normalized, left_index=True, right_index=True )
final = final[['DM1','DM2']]
#display(final.head())

# Step 2.1 : Calculate total score
final['Total Score'] = final.sum(1)
#display(final.head())

# Step 2.2 : Rank stocks by total score (sorting)
final = final.sort_values('Total Score', ascending=False)
#display(final.head())


# Step 2.2 : Average of scores of DM1, DM2
final['Average'] = final[['DM1', 'DM2']].mean(1)
#display(final.head())


# Step 2.3 : Standard deviation of scores of DM1, DM2
final['St. dev'] = final[['DM1', 'DM2']].std(1)

#######################DROP STOCKS#######################
TH_std = 0.8

# Step 3.1 : Droping stocks based on their bad total score
final = final[final['Total Score'] >= 0]

# Step 3.2 : Droping stocks based on the standard deviation threshold
final = final[final['St. dev'] <= TH_std]


# Step 4.1 : Calculate consensus & total % consensus
def consensus(data):
  return 100 - (100 * data['St. dev'] / TH_std)

# Step 4.2 : Droping stocks based on their bad consensus performance
TH_con = 15

final['Standard Consensus'] = final[['DM1', 'DM2', 'St. dev']].apply(consensus, axis = 1)
final = final[final['Standard Consensus'] >= TH_con]
final = final.sort_values('Standard Consensus', ascending=False)
print(final.shape)
display(final)
print("Final Average Consensus for DM1, DM2 is : " + str(round(final['Standard Consensus'].mean(0), 2)) + "%")

# download consensus table
final.to_csv('h.csv')
create_download_link(filename = 'h.csv')

(22, 6)


,DM1,DM2,Total Score,Average,St. dev,Standard Consensus
Symbol,,,,,,
TXN,14.604107,14.534014,29.138121,14.569061,0.049563,93.804643
OTEX,19.417729,19.513887,38.931617,19.465808,0.067994,91.500772
AMAT,7.792784,7.663033,15.455817,7.727909,0.091748,88.531498
UBNT,39.425744,39.291003,78.716747,39.358374,0.095277,88.090401
AMSWA,13.476771,13.692423,27.169194,13.584597,0.152489,80.938835
TTEC,25.567020,25.329434,50.896454,25.448227,0.167999,79.000134
AVGO,20.341370,20.073766,40.415136,20.207568,0.189225,76.346897
AAPL,101.679142,101.370049,203.049190,101.524595,0.218562,72.679757
QCOM,33.416355,33.047022,66.463378,33.231689,0.261158,67.355254


Final Average Consensus for DM1, DM2 is : 58.13%


In [25]:
#######################################################
#      D. Apply TOPSIS, ELECTRE with veto, PROMETHE   #
#######################################################
# > In this sectiton we apply all the above metthodologies to reach a single (or a pool)
# of stocks
# > Depsite the use of our own data to conclude upon the DM2 values, for this section
# we will proceed with actual data we already have from a trust worthy source
# investing.com
stock_tickers = list(final.index)

# Step 1 :  Keep the 22 stocks
tickers_exclude = list(set(list(stocks_investingcom.index)) - set(stock_tickers))
stocks = stocks_investingcom.drop(tickers_exclude)
stocks = stocks.reindex(final.index)

# Step 2 :  Normalize again with different weights this time since our perspective
# have changed. In the previous setction we give greater weights to our risk metrics
# and YTD calculated by our own analysis so that we could have a fair enough DM2
# results.
# Market Cap : 2.5 %
# P/E Ratio: 10%
# Revenue: 15%
# Average Vol: 2.5%
# EPS: 15%
# Beta: 12.5%
# YTD: 10%
# 1 Year: 10%
# 3 Year: 2.5%
# Weekly: 12.5%
# Monthly: 7.5%
weights = [ 0.025, 0.1, 0.15, 0.025, 0.15, 0.125, 0.1, 0.1, 0.025, 0.125, 0.075]

stocks_normalized = normalization(stocks,1,weights)
del stocks_normalized['DM1']
del stocks_normalized['Name']
display(stocks)



,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
Symbol,,,,,,,,,,,,
TXN,Texas Instruments,1.046900e+11,20.16,1.559000e+10,4850000.0,5.51,1.22,17.39,-2.90,78.89,4,4
OTEX,Open Text,1.110000e+10,40.51,2.880000e+09,499120.0,1.02,0.48,25.43,13.71,35.62,4,4
AMAT,Applied Materials,3.971000e+10,11.86,1.577000e+10,9230000.0,3.57,1.63,29.23,-12.91,79.05,4,3
UBNT,Ubiquiti,9.220000e+09,29.86,1.140000e+09,449380.0,4.44,1.34,30.95,51.18,225.04,2,4
AMSWA,American Software,4.432200e+08,73.81,1.117900e+08,66460.0,0.19,0.69,36.17,-6.13,42.16,4,4
TTEC,TTEC,2.070000e+09,41.87,1.530000e+09,84980.0,1.08,0.68,57.51,20.81,63.46,4,4
AVGO,Broadcom,1.102900e+11,33.95,2.131000e+10,2790000.0,8.19,0.91,7.86,4.99,74.77,2,4
AAPL,Apple,9.124400e+11,17.01,2.584900e+11,29290000.0,11.67,1.23,25.83,6.42,108.71,4,4
QCOM,Qualcomm,8.680000e+10,39.71,2.123000e+10,21120000.0,1.81,1.61,24.78,20.79,31.77,3,4


In [26]:
##################
#   1. TOPSIS    #
##################
# 1. The basic principle of the TOPSIS method is that the chosen alternative 
#    should have the shortest distance from the positive ideal solution (PIS) 
#    and the farthest distance from the negative ideal solution (NIS). It is an 
#    effective method to determine the total ranking order of decision alternatives.
# 2. TOPSIS method are used to derive the closeness coefficient and the outranking 
#    index of each stock, respectively. Based on the closeness coefficient, the 
#    outranking index, and selection threshold, we can easily obtain three type of 
#    the investment ratio in accordance with different investment preference of final decision-maker. 
#    It is a reasonable way in real decision environment

# Step 1 : Ideal and anti-ideal solutions
topsis_ideal = pd.DataFrame(index = ['Positive ideal solution','Negative ideal solution'], columns = list(stocks_normalized.columns))
topsis_ideal.loc['Positive ideal solution'] = stocks_normalized.max()
topsis_ideal.loc['Negative ideal solution'] = stocks_normalized.min()

# Step 2 : Calculation of the Separation Measures D+, D-. This step is about the calculation 
# of the distances of each alternative from the ideal solution as:
#########################################
#                 _______________       #
#                | --                   #
#                | \               2    #
#        D_i* =  | /  (v_ij - v_j*)     #
#               \| --                   #
#                                       #
#########################################
#########################################
#                 _______________       #
#                | --                   #
#           _    | \             _ 2    #
#        D_i  =  | /  (v_ij - v_j )     #
#               \| --                   #
#                                       #
#########################################
stocks_topsis = stocks_normalized.copy()

# Step 2.1 : Calculate D+, D-
def D_plus(data):
  return np.sqrt( sum((data - topsis_ideal.loc['Positive ideal solution'])**2)  )

def D_minus(data):
  return np.sqrt( sum((data - topsis_ideal.loc['Negative ideal solution'])**2)  )

D_plus = stocks_topsis.apply(D_plus, axis = 1)
D_minus = stocks_topsis.apply(D_minus, axis = 1)


stocks_topsis['D_plus'] = D_plus
stocks_topsis['D_minus'] = D_minus


# Step 3 : Calculation of the Relative Closeness to the Ideal Solution 
# The relative closeness C_i* is always between 0 and 1 and an alternative is 
# best when it is closer to 1. It is calculated for each alternative and is defined as
#########################################
#                     _                 #
#                  D_i                  #
#                                       #
#        C_i* = ----------              #
#                         _             #
#               D_i* + D_i              #
#                                       #
#########################################
def C(data):
  return data['D_minus'] / (data['D_plus'] + data['D_minus'])
  
stocks_topsis['C_closeness'] =  stocks_topsis.apply(C, axis = 1)
display(stocks_topsis )

# Step 4 : Step 6. Ranking of the Preference Order Finally, the alternatives 
# are ranked from best (higher relative closeness value) to worst. 
# The best alternative and the solution to the problem is on the top of the list.
TH_topsis = 0.25
stocks_topsis = stocks_topsis.sort_values('C_closeness', ascending=False)
stocks_topsis = stocks_topsis[stocks_topsis['C_closeness'] >= TH_topsis]

display(stocks_topsis)

# Step 5 :  Print company names we will invest in

companynames_topsis = [stocks_tickers.loc[stocks_tickers['Symbol'] == i, 'Name'].item() for i in list(stocks_topsis.index) ]
print("We will invest in the following companies :")
print(companynames_topsis)

display(stocks.loc[list(stocks_topsis.index)])


,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly,D_plus,D_minus,C_closeness
Symbol,,,,,,,,,,,,,,
TXN,0.754252,0.614305,2.124190,0.657164,2.170438,2.194766,0.976380,-0.694651,1.226219,3.159194,0.217571,43.686894,4.554020,0.094402
OTEX,0.079971,1.234400,0.392410,0.067630,0.401787,0.863515,1.427795,3.284022,0.553656,3.159194,0.217571,43.054234,6.719751,0.135005
AMAT,0.286096,0.361392,2.148715,1.250645,1.406255,2.932352,1.641150,-3.092394,1.228706,3.159194,0.163178,45.209548,4.088907,0.082942
UBNT,0.066427,0.909879,0.155329,0.060890,1.748956,2.410645,1.737721,12.259392,3.497887,1.579597,0.217571,38.990413,15.871662,0.289301
AMSWA,0.003193,2.249101,0.015232,0.009005,0.074843,1.241302,2.030803,-1.468348,0.655310,3.159194,0.217571,45.979810,3.375561,0.068393
TTEC,0.014914,1.275842,0.208468,0.011515,0.425422,1.223312,3.228961,4.984719,0.986384,3.159194,0.217571,42.163467,8.767989,0.172153
AVGO,0.794598,1.034507,2.903559,0.378039,3.226114,1.637080,0.441308,1.195279,1.162180,1.579597,0.217571,41.996721,6.239085,0.129346
AAPL,6.573785,0.518320,35.220131,3.968731,4.596918,2.212756,1.450253,1.537813,1.689723,3.159194,0.217571,25.198411,36.701146,0.592915
QCOM,0.625361,1.210023,2.892659,2.861714,0.712975,2.896372,1.391300,4.979929,0.493814,2.369396,0.217571,39.802336,9.423257,0.191430


,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly,D_plus,D_minus,C_closeness
Symbol,,,,,,,,,,,,,,
AAPL,6.573785,0.518320,35.220131,3.968731,4.596918,2.212756,1.450253,1.537813,1.689723,3.159194,0.217571,25.198411,36.701146,0.592915
OLED,0.062536,3.207125,0.045669,0.099243,0.689341,2.716473,5.423149,26.145225,2.562640,3.159194,0.217571,36.325830,29.972628,0.452086
MSFT,7.204622,0.918411,16.651523,3.209943,1.764712,2.212756,1.816325,7.655533,2.618752,3.159194,0.217571,26.771686,21.640885,0.447010
XLNX,0.203819,0.999770,0.416935,0.590770,1.343230,2.194766,1.700665,14.772112,2.155092,1.579597,0.217571,37.908481,18.105980,0.323238
BRKR,0.053386,1.232877,0.262969,0.109944,0.460873,2.284716,3.270509,13.301368,1.449111,3.159194,0.217571,38.534862,16.827787,0.303956
UBNT,0.066427,0.909879,0.155329,0.060890,1.748956,2.410645,1.737721,12.259392,3.497887,1.579597,0.217571,38.990413,15.871662,0.289301
CSCO,1.735738,0.623752,6.992522,2.718086,1.079311,2.140797,1.661362,6.795603,1.476157,3.159194,0.217571,35.252597,12.825610,0.266766


We will invest in the following companies :
['Apple', 'Universal Display', 'Microsoft', 'Xilinx', 'Bruker', 'Ubiquiti', 'Cisco']


,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
Symbol,,,,,,,,,,,,
AAPL,Apple,9.124400e+11,17.01,2.584900e+11,29290000.0,11.67,1.23,25.83,6.42,108.71,4,4
OLED,Universal Display,8.680000e+09,105.25,3.351800e+08,732430.0,1.75,1.51,96.59,109.15,164.87,4,4
MSFT,Microsoft,1.000000e+12,30.14,1.222100e+11,23690000.0,4.48,1.23,32.35,31.96,168.48,4,4
XLNX,Xilinx,2.829000e+10,32.81,3.060000e+09,4360000.0,3.41,1.22,30.29,61.67,138.65,2,4
BRKR,Bruker,7.410000e+09,40.46,1.930000e+09,811410.0,1.17,1.27,58.25,55.53,93.23,4,4
UBNT,Ubiquiti,9.220000e+09,29.86,1.140000e+09,449380.0,4.44,1.34,30.95,51.18,225.04,2,4
CSCO,Cisco,2.409200e+11,20.47,5.132000e+10,20060000.0,2.74,1.19,29.59,28.37,94.97,4,4


In [27]:
#######################################
#   2. ELECTRE I with & without veto  #
#######################################
# Step 1 : Define the necessary DataFrames of agreement & disagreement
stocks_electreI = stocks.copy()
del stocks_electreI['Name']

# Step 2 : Define veto thresholds. The veto thresholds express the power attributed to a given criteria to be against the assertion “a outranks b”, when
# the difference in the evaluation between g(b) and g(a) is greater than this threshold.
veto = [10000000000000000000000000, 15,10000000000000000000000000,10000000000000000000000000,5,0.6,25, 45,80,10000000000000000000000000,10000000000000000000000000]

eledctrI_agree = pd.DataFrame(index = stocks_electreI.index, columns = stocks_electreI.index)
eledctrI_disagree = eledctrI_agree.copy()
eledctrI_disagree_veto = eledctrI_agree.copy()

# Step 2 : Calculate agreement & disagreemnt table
# define delta for disagreement

c = stocks_electreI.max()
d = stocks_electreI.min()
delta = max(c - d)

for i in eledctrI_agree.columns:
  for j in eledctrI_agree.columns:
    a = stocks_electreI.loc[i]
    b = stocks_electreI.loc[j]
    # Step 2.1 : agreement cell calculation
    eledctrI_agree[j].loc[i] =  sum([ weights[i] for i in range(len((a-b))) if (a-b)[i] >=0 ])
    
    # Step 2.2 : disagreement cell calculation
    # with veto
    dis = (b - a) - veto
    dis_result = 1 if any(i >= 0 for i in dis) else 0
    eledctrI_disagree_veto[j].loc[i] = dis_result
    # without veto
    eledctrI_disagree[j].loc[i] = max(dis) / delta
    
display(eledctrI_agree)
display(eledctrI_disagree)
display(eledctrI_disagree_veto)


#display(eledctrI_agree)
#display(eledctrI_disagree_veto)

# Step 3.1.1 : FIND kernel after applying ELECTRE withtout veto
# Start with c = 1.0, d = 0 as initial threshold values for 
# agreement and disagreement. Relaxing those thresholds until we
# include at least 5 stocks in our kernel
c = 1.0
d = 0.0

kernel = list(eledctrI_agree.columns)
changes = {}

for k in range(1000000000):
  length = len(kernel)
  for i in eledctrI_agree.columns:
    for j in eledctrI_agree.columns:
      if i == j: continue
      if(eledctrI_agree[j].loc[i]  >= c and eledctrI_disagree[j].loc[i] <= d):
        if j in kernel:
          kernel.remove(j)
  a = kernel.copy()
  if(len(kernel) < length): changes.update({(c,d):a})
  if(len(kernel) < 6): break
  c -= 0.01
  d += 0.01
  
# Step 3.1.2 :  Print company names we will invest in
companynames_electreI = [stocks_tickers.loc[stocks_tickers['Symbol'] == i, 'Name'].item() for i in kernel ]
print("Electre I withtout veto : We will invest in the following companies :")
print(companynames_electreI)
print()
print(changes)
print()
display(stocks.loc[kernel])

# Step 3.2 : FIND kernel after applying ELECTRE withtout veto
kernel_veto = list(eledctrI_agree.columns)
changes_veto = {}
s = 1
for k in range(100000000):
  length = len(kernel_veto)
  for i in eledctrI_agree.columns:
    for j in eledctrI_agree.columns:
      if i == j: continue
      if(eledctrI_agree[j].loc[i]  >= s and eledctrI_disagree_veto[j].loc[i] == 0):
        if j in kernel_veto:
          kernel_veto.remove(j)
  a = kernel_veto.copy()
  if(len(kernel_veto) < length): changes_veto.update({(s):a})
  if(len(kernel_veto) < 6): break
  s -= 0.01
  
  
# Step 3.3.2 :  Print company names we will invest in
companynames_electreI = [stocks_tickers.loc[stocks_tickers['Symbol'] == i, 'Name'].item() for i in kernel_veto ]
print("Electre with veto : We will invest in the following companies :")
print(companynames_electreI)
print()
print(changes_veto)

display(stocks.loc[kernel_veto])



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Symbol,TXN,OTEX,AMAT,UBNT,AMSWA,TTEC,AVGO,AAPL,QCOM,XLNX,ESLT,TER,CSCO,SSNC,CDW,MSFT,MANT,GRMN,ADI,JKHY,BRKR,OLED
Symbol,,,,,,,,,,,,,,,,,,,,,,
TXN,1,0.7,0.575,0.55,0.8,0.7,0.475,0.3,0.4,0.675,0.7,0.65,0.475,0.55,0.525,0.35,0.7,0.675,0.55,0.8,0.55,0.55
OTEX,0.5,1,0.4,0.5,0.65,0.4,0.5,0.4,0.425,0.3,0.35,0.475,0.3,0.45,0.3,0.3,0.6,0.3,0.4,0.7,0.475,0.375
AMAT,0.55,0.725,1,0.45,0.625,0.625,0.4,0.35,0.525,0.6,0.475,0.6,0.4,0.7,0.3,0.25,0.725,0.55,0.525,0.65,0.6,0.6
UBNT,0.525,0.575,0.55,1,0.675,0.525,0.55,0.525,0.45,0.6,0.375,0.475,0.675,0.7,0.675,0.325,0.625,0.675,0.55,0.725,0.4,0.425
AMSWA,0.4,0.55,0.5,0.4,1,0.425,0.4,0.4,0.425,0.4,0.325,0.3,0.4,0.3,0.4,0.4,0.4,0.4,0.4,0.4,0.3,0.2
TTEC,0.5,0.8,0.5,0.55,0.775,1,0.5,0.5,0.525,0.4,0.425,0.5,0.4,0.55,0.4,0.4,0.5,0.4,0.5,0.5,0.3,0.35
AVGO,0.6,0.575,0.6,0.65,0.675,0.575,1,0.175,0.425,0.625,0.675,0.525,0.325,0.55,0.525,0.325,0.525,0.525,0.525,0.675,0.425,0.425
AAPL,0.9,0.8,0.775,0.55,0.8,0.7,0.9,1,0.675,0.675,0.7,0.55,0.7,0.575,0.675,0.65,0.8,0.7,0.575,0.9,0.575,0.55
QCOM,0.675,0.65,0.475,0.625,0.65,0.55,0.65,0.4,1,0.625,0.5,0.6,0.325,0.775,0.5,0.3,0.7,0.5,0.6,0.825,0.55,0.55


Symbol,TXN,OTEX,AMAT,UBNT,AMSWA,TTEC,AVGO,AAPL,QCOM,XLNX,ESLT,TER,CSCO,SSNC,CDW,MSFT,MANT,GRMN,ADI,JKHY,BRKR,OLED
Symbol,,,,,,,,,,,,,,,,,,,,,,
TXN,-6.00266e-13,5.35237e-12,-1.90084e-13,6.61793e-11,3.86671e-11,1.51267e-11,-9.10404e-13,1.16051e-12,4.55202e-12,1.95787e-11,-9.90439e-13,3.53157e-12,-6.30279e-13,8.28567e-11,-7.70341e-13,9.59425e-12,-8.8039e-13,-8.40372e-13,-4.20186e-13,2.39106e-12,1.5867e-11,7.01211e-11
OTEX,1.40062e-13,-6.00266e-13,5.50244e-13,1.09469e-10,1.83081e-11,7.08314e-12,2.17096e-12,5.65251e-12,5.30235e-13,2.30402e-11,-2.50111e-13,1.90885e-11,1.10049e-13,6.24977e-11,3.97076e-11,5.28834e-11,-1.40062e-13,-1.00044e-13,3.20142e-13,-1.40062e-13,7.82347e-12,5.04624e-11
AMAT,-1.01045e-12,1.36561e-11,-6.00266e-13,6.60193e-11,4.69708e-11,1.50167e-11,7.09314e-12,3.10137e-12,1.28557e-11,2.95931e-11,5.31235e-12,-6.80302e-13,-1.04046e-12,9.11604e-11,-1.18052e-12,9.43418e-12,3.99177e-12,-5.60248e-13,-6.70297e-13,1.06947e-11,2.34504e-11,7.84248e-11
UBNT,-7.20319e-13,-1.46065e-12,-3.10137e-13,-6.00266e-13,2.89628e-11,1.56069e-12,-1.03046e-12,2.23099e-12,-3.30146e-13,-7.20319e-13,-1.11049e-12,-3.90173e-13,-7.50333e-13,7.31524e-11,-8.90395e-13,-7.10315e-13,-1.00044e-12,-9.60426e-13,-5.40239e-13,-1.00044e-12,2.30102e-12,6.04168e-11
AMSWA,3.20142e-13,-8.10359e-13,3.40151e-13,1.02926e-10,-6.00266e-13,-6.1027e-13,3.00133e-12,6.48287e-12,3.20142e-13,2.28101e-11,-3.40151e-13,1.25456e-11,-1.00044e-13,2.91829e-11,3.31647e-11,4.63405e-11,-3.50155e-13,-3.10137e-13,1.10049e-13,-3.50155e-13,1.66674e-11,7.03112e-11
TTEC,-6.00266e-14,-8.00355e-13,3.50155e-13,8.16162e-11,1.69475e-11,-6.00266e-13,2.11094e-12,5.59248e-12,3.30146e-13,-6.00266e-14,-4.502e-13,2.7012e-13,-9.00399e-14,6.11371e-11,1.18553e-11,2.50311e-11,-3.40151e-13,-3.00133e-13,1.20053e-13,-3.40151e-13,-1.00044e-14,4.84015e-11
AVGO,-2.90129e-13,-1.03046e-12,1.20053e-13,7.03012e-11,2.4871e-11,2.46609e-11,-6.00266e-13,-2.80124e-13,1.00044e-13,1.16852e-11,5.52245e-12,1.30658e-11,-3.20142e-13,6.90606e-11,5.40239e-13,1.37161e-11,-5.70253e-13,-5.30235e-13,-1.10049e-13,-5.70253e-13,2.54013e-11,6.37583e-11
AAPL,-6.1027e-13,8.50377e-12,-2.00089e-13,3.63461e-11,4.18185e-11,9.86437e-12,1.94086e-12,-6.00266e-13,7.70341e-12,1.02545e-11,1.60071e-13,-2.80124e-13,-6.40284e-13,8.60081e-11,-7.80346e-13,-6.00266e-13,-8.90395e-13,-8.50377e-13,-4.30191e-13,5.54246e-12,8.45375e-12,7.32725e-11
QCOM,-9.90439e-13,-1.73077e-12,-5.80257e-13,1.1332e-10,1.91085e-11,7.73343e-12,1.38061e-12,4.86216e-12,-6.00266e-13,2.68919e-11,-1.38061e-12,2.29402e-11,-1.02045e-12,6.32981e-11,4.35593e-11,5.67351e-11,-1.27056e-12,-1.23055e-12,-8.10359e-13,-1.27056e-12,8.47376e-12,5.31235e-11


Symbol,TXN,OTEX,AMAT,UBNT,AMSWA,TTEC,AVGO,AAPL,QCOM,XLNX,ESLT,TER,CSCO,SSNC,CDW,MSFT,MANT,GRMN,ADI,JKHY,BRKR,OLED
Symbol,,,,,,,,,,,,,,,,,,,,,,
TXN,0,1,0,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,1,1,1
OTEX,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,1,0,1,1
AMAT,0,1,0,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,1,1,1
UBNT,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1
AMSWA,1,0,1,1,0,0,1,1,1,1,0,1,0,1,1,1,0,0,1,0,1,1
TTEC,0,0,1,1,1,0,1,1,1,0,0,1,0,1,1,1,0,0,1,0,0,1
AVGO,0,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,0,0,0,0,1,1
AAPL,0,1,0,1,1,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1
QCOM,0,0,0,1,1,1,1,1,0,1,0,1,0,1,1,1,0,0,0,0,1,1


Electre I withtout veto : We will invest in the following companies :
['Ubiquiti', 'Apple', 'Elbit Systems', 'Microsoft', 'Universal Display']

{(1.0, 0.0): ['TXN', 'OTEX', 'AMAT', 'UBNT', 'TTEC', 'AVGO', 'AAPL', 'QCOM', 'XLNX', 'ESLT', 'TER', 'SSNC', 'MSFT', 'MANT', 'ADI', 'JKHY', 'BRKR', 'OLED'], (0.8999999999999999, 0.09999999999999999): ['AMAT', 'UBNT', 'AAPL', 'QCOM', 'XLNX', 'ESLT', 'TER', 'SSNC', 'MSFT', 'ADI', 'BRKR', 'OLED'], (0.8699999999999999, 0.12999999999999998): ['UBNT', 'AAPL', 'QCOM', 'XLNX', 'ESLT', 'TER', 'SSNC', 'MSFT', 'BRKR', 'OLED'], (0.8199999999999998, 0.18000000000000002): ['UBNT', 'AAPL', 'QCOM', 'XLNX', 'ESLT', 'TER', 'SSNC', 'MSFT', 'OLED'], (0.7999999999999998, 0.20000000000000004): ['UBNT', 'AAPL', 'QCOM', 'ESLT', 'TER', 'SSNC', 'MSFT', 'OLED'], (0.7699999999999998, 0.23000000000000007): ['UBNT', 'AAPL', 'ESLT', 'MSFT', 'OLED']}



,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
Symbol,,,,,,,,,,,,
UBNT,Ubiquiti,9.220000e+09,29.86,1.140000e+09,449380.0,4.44,1.34,30.95,51.18,225.04,2,4
AAPL,Apple,9.124400e+11,17.01,2.584900e+11,29290000.0,11.67,1.23,25.83,6.42,108.71,4,4
ESLT,Elbit Systems,6.770000e+09,32.17,4.710000e+09,12680.0,4.85,0.83,38.38,30.99,70.87,4,4
MSFT,Microsoft,1.000000e+12,30.14,1.222100e+11,23690000.0,4.48,1.23,32.35,31.96,168.48,4,4
OLED,Universal Display,8.680000e+09,105.25,3.351800e+08,732430.0,1.75,1.51,96.59,109.15,164.87,4,4


Electre with veto : We will invest in the following companies :
['Apple', 'Elbit Systems', 'SS&Cs', 'Microsoft', 'Universal Display']

{1: ['TXN', 'OTEX', 'AMAT', 'UBNT', 'TTEC', 'AVGO', 'AAPL', 'QCOM', 'XLNX', 'ESLT', 'TER', 'SSNC', 'MSFT', 'MANT', 'ADI', 'JKHY', 'BRKR', 'OLED'], 0.8999999999999999: ['AMAT', 'UBNT', 'AVGO', 'AAPL', 'QCOM', 'XLNX', 'ESLT', 'TER', 'SSNC', 'MSFT', 'ADI', 'BRKR', 'OLED'], 0.8699999999999999: ['UBNT', 'AVGO', 'AAPL', 'QCOM', 'XLNX', 'ESLT', 'TER', 'SSNC', 'MSFT', 'BRKR', 'OLED'], 0.8199999999999998: ['UBNT', 'AVGO', 'AAPL', 'QCOM', 'XLNX', 'ESLT', 'TER', 'SSNC', 'MSFT', 'OLED'], 0.7999999999999998: ['UBNT', 'AVGO', 'AAPL', 'QCOM', 'ESLT', 'TER', 'SSNC', 'MSFT', 'OLED'], 0.7699999999999998: ['UBNT', 'AVGO', 'AAPL', 'ESLT', 'SSNC', 'MSFT', 'OLED'], 0.7499999999999998: ['AAPL', 'ESLT', 'SSNC', 'MSFT', 'OLED']}


,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
Symbol,,,,,,,,,,,,
AAPL,Apple,9.124400e+11,17.01,2.584900e+11,29290000.0,11.67,1.23,25.83,6.42,108.71,4,4
ESLT,Elbit Systems,6.770000e+09,32.17,4.710000e+09,12680.0,4.85,0.83,38.38,30.99,70.87,4,4
SSNC,SS&Cs,1.472000e+10,117.98,4.140000e+09,1360000.0,0.49,1.29,28.77,6.49,98.06,2,4
MSFT,Microsoft,1.000000e+12,30.14,1.222100e+11,23690000.0,4.48,1.23,32.35,31.96,168.48,4,4
OLED,Universal Display,8.680000e+09,105.25,3.351800e+08,732430.0,1.75,1.51,96.59,109.15,164.87,4,4


In [38]:
######################
#   3. PROMETTHEE    #
######################
stocks_promethee = stocks.copy()
promethee_flows = stocks.copy()
del stocks_promethee['Name']
del promethee_flows['Name']
# threshold of absolute preference, threshold of indifference and type for each criterion
p_q_type = {'Market Cap':(10**9, 10**11, 'typeV'), 'P/E Ratio': (15, 35,'typeV'), 'Revenue':(10**8, 10**10, 'typeV'), 'Average Vol. (3m)':(10**7, 10**8, 'typeV'), 
            'EPS':(1,6, 'typeV'), 'Beta':(0,0.6, 'typeV'), 'YTD':(5, 25, 'typeV'), '1 Year':(5, 45, 'typeV'), '3 Years':(10, 75,'typeV'), 'Weekly':(0,1,'typeI'), 'Monthly':(0,1,'typeI')}

def typeV(d, q, p):
  if(d <= q): return 0
  elif (d > p): return 1
  else: return (d - q)/ (p - q)

def typeI(d):
  return 1 if d > 0 else 0
  

for crit in stocks_promethee.columns:
  print(crit)
  promethee_preferences_crit = pd.DataFrame(index = stocks_promethee.index, columns = stocks_promethee.index)
  for i in promethee_preferences_crit.columns:
    for j in promethee_preferences_crit.columns:
      # Step 1 : Calculate the differences between the evaluations of the stocks on the specific criterion
      diff = stocks_promethee[crit].loc[j] - stocks_promethee[crit].loc[i]
      # Step 2 : Calculate pairwise comparison 
      # Use as a criterion :  linear (type V) preference functions are best suited for quantitative criteria (e.g. prices, costs, power, ...)
     
      q,p,type_i = p_q_type[crit]
      
      if(type_i == 'typeV'):
        promethee_preferences_crit[j].loc[i] = typeV(diff, q, p)
      else:
        promethee_preferences_crit[j].loc[i] = typeI(diff)
   
 
  # Step 3 : Positive, negative, and net flows for the investment criterion
  length = len(promethee_preferences_crit) - 1
  promethee_preferences_crit['Positive Flow'] = promethee_preferences_crit.sum(1) / length
  promethee_preferences_crit['Negative Flow'] = promethee_preferences_crit.sum(0) / length
  promethee_preferences_crit['Net Flow'] = promethee_preferences_crit['Negative Flow'] - promethee_preferences_crit['Positive Flow'] 
  
  # Step 4 : Append flows for this criterion in the final dataframe
  promethee_flows[crit] =  promethee_preferences_crit['Net Flow']
  
display(promethee_flows)

# Step 5 : Calculate weighted global net flow
for i in promethee_flows.index:
  promethee_flows.loc[i] = promethee_flows.loc[i] * weights
  
promethee_flows['Net Flow'] = promethee_flows.sum(1)
promethee_flows = promethee_flows.sort_values('Net Flow', ascending = False)
display(promethee_flows)

  
# Step 6 :  Print company names we will invest in
kernel_promethee = list(promethee_flows.head(7).index)
companynames_promethee = [stocks_tickers.loc[stocks_tickers['Symbol'] == i, 'Name'].item() for i in kernel_promethee ]
print("We will invest in the following companies :")
print(companynames_promethee)

display(stocks.loc[kernel_promethee])


Market Cap


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


P/E Ratio
Revenue
Average Vol. (3m)
EPS
Beta
YTD
1 Year
3 Years
Weekly
Monthly


,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
Symbol,,,,,,,,,,,
TXN,0.550495,-0.200714,0.462290,-0.018392,0.267143,0.119841,-0.418286,-0.466893,-0.214542,0.285714,0.095238
OTEX,-0.292570,-0.035595,-0.357561,-0.027600,-0.314667,-0.871429,-0.151857,-0.174679,-0.647297,0.285714,0.095238
AMAT,-0.036154,-0.367667,0.466522,-0.009122,0.013619,0.704762,-0.089667,-0.659690,-0.213018,0.285714,-0.952381
UBNT,-0.307327,-0.135714,-0.468961,-0.027705,0.114381,0.311111,-0.056690,0.589917,0.971766,-0.809524,0.095238
AMSWA,-0.374761,0.800595,-0.536277,-0.028515,-0.428667,-0.691270,0.086429,-0.528417,-0.584725,0.285714,0.095238
TTEC,-0.362661,-0.008071,-0.443901,-0.028476,-0.307238,-0.703175,0.802524,-0.036976,-0.358564,0.285714,0.095238
AVGO,0.587128,-0.121357,0.599230,-0.022751,0.729333,-0.389683,-0.769167,-0.328417,-0.253780,-0.809524,0.095238
AAPL,0.910746,-0.245119,1.000000,0.166544,0.976000,0.136508,-0.144238,-0.306000,0.115597,0.285714,0.095238
QCOM,0.392283,-0.050833,0.598076,0.088734,-0.220667,0.682540,-0.165452,-0.037357,-0.682828,-0.523810,0.095238


,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly,Net Flow
Symbol,,,,,,,,,,,,
AAPL,0.022769,-0.024512,0.150000,0.004164,0.146400,0.017063,-0.014424,-0.030600,0.002890,0.035714,0.007143,0.316607
OLED,-0.007784,0.094390,-0.078329,-0.000678,-0.034129,0.070139,0.100000,0.100000,0.018078,0.035714,0.007143,0.304545
MSFT,0.024850,-0.013505,0.135714,0.002830,0.017900,0.017063,-0.002617,0.018575,0.018662,0.035714,0.007143,0.262331
CSCO,0.020238,-0.019702,0.121429,0.001966,-0.014457,0.008730,-0.008367,0.011454,-0.001355,0.035714,0.007143,0.162793
CDW,-0.006407,-0.016055,0.073131,-0.000680,0.016086,-0.020734,-0.008417,0.003842,0.016444,0.035714,0.007143,0.100067
BRKR,-0.008010,-0.003655,-0.062826,-0.000673,-0.044543,0.025397,0.080781,0.065057,-0.001860,0.035714,0.007143,0.092525
TXN,0.013762,-0.020071,0.069343,-0.000460,0.040071,0.014980,-0.041829,-0.046689,-0.005364,0.035714,0.007143,0.066602
QCOM,0.009807,-0.005083,0.089711,0.002218,-0.033100,0.085317,-0.016545,-0.003736,-0.017071,-0.065476,0.007143,0.053186
TER,-0.007917,-0.020095,-0.061268,-0.000602,-0.024329,0.076687,0.047531,-0.011420,0.011171,0.035714,0.007143,0.052614


We will invest in the following companies :
['Apple', 'Universal Display', 'Microsoft', 'Cisco', 'CDW Corp', 'Bruker', 'Texas Instruments']


,Name,Market Cap,P/E Ratio,Revenue,Average Vol. (3m),EPS,Beta,YTD,1 Year,3 Years,Weekly,Monthly
Symbol,,,,,,,,,,,,
AAPL,Apple,9.124400e+11,17.01,2.584900e+11,29290000.0,11.67,1.23,25.83,6.42,108.71,4,4
OLED,Universal Display,8.680000e+09,105.25,3.351800e+08,732430.0,1.75,1.51,96.59,109.15,164.87,4,4
MSFT,Microsoft,1.000000e+12,30.14,1.222100e+11,23690000.0,4.48,1.23,32.35,31.96,168.48,4,4
CSCO,Cisco,2.409200e+11,20.47,5.132000e+10,20060000.0,2.74,1.19,29.59,28.37,94.97,4,4
CDW,CDW Corp,1.534000e+10,23.78,1.659000e+10,684910.0,4.38,1.05,29.56,24.58,155.31,4,4
BRKR,Bruker,7.410000e+09,40.46,1.930000e+09,811410.0,1.17,1.27,58.25,55.53,93.23,4,4
TXN,Texas Instruments,1.046900e+11,20.16,1.559000e+10,4850000.0,5.51,1.22,17.39,-2.90,78.89,4,4
